In [1]:
!git clone https://github.com/ChuloIva/COT-steering.git
%cd COT-steering
!git checkout try3

Cloning into 'COT-steering'...
remote: Enumerating objects: 666, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 666 (delta 54), reused 119 (delta 34), pack-reused 521 (from 1)
Receiving objects: 100% (666/666), 25.23 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (334/334), done.
/content/COT-steering
Branch 'try3' set up to track remote branch 'try3' from 'origin'.
Switched to a new branch 'try3'


In [2]:
!pip install torch transformers nnsight numpy tqdm matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## we need to install this just because it's in the repo, we don't actually use it here

In [4]:
!pip install dotenv openai anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 13.3 MB/s eta 0:00:00


In [6]:
import torch
from nnsight import LanguageModel
from collections import defaultdict
import json
from tqdm import tqdm
# Import helper functions from the original repo's utils
# (You'd copy get_label_positions and other necessary functions into your project)
from utils import get_label_positions, process_saved_responses_batch

## Some edits to functions used

In [7]:
def get_batched_message_ids(tokenizer, messages_list):
    """Tokenizes a batch of messages with padding."""
    # This ensures all sequences in the batch have the same length.
    return tokenizer(
        messages_list,
        padding='longest',
        return_tensors="pt"
    ).to("cuda")

def process_saved_responses_batch(responses_list, tokenizer, model):
    """
    Correctly gets layer activations for a batch of saved responses.
    This version ensures the output tensor has the correct shape.
    """
    tokenized_responses = get_batched_message_ids(tokenizer, responses_list)
    input_ids = tokenized_responses['input_ids']
    attention_mask = tokenized_responses['attention_mask']

    # Use model.trace to run a forward pass and capture activations
    with model.trace({"input_ids": input_ids, "attention_mask": attention_mask}) as tracer:
        # Capture the output of every layer
        layer_proxies = [
            model.model.layers[i].output[0].save()
            for i in range(model.config.num_hidden_layers)
        ]

    # After the forward pass, retrieve the actual tensors from the proxies
    # Each tensor in this list has shape [batch_size, seq_len, hidden_size]
    layer_outputs_tensors = [p.value.cpu().detach().to(torch.float32) for p in layer_proxies]

    batch_layer_outputs = []
    num_examples_in_batch = input_ids.shape[0]

    for i in range(num_examples_in_batch):
        # Find the actual length of the response, ignoring padding
        # The sum of the attention mask gives the non-padded length
        non_padded_length = int(attention_mask[i].sum())

        # For this single example, stack the activations from all layers.
        # This is the critical step for getting the dimensions right.
        example_outputs = torch.stack([
            # For each layer's output tensor...
            layer_tensors[i, :non_padded_length, :]
            # ...take the i-th example and slice it to its actual length.
            for layer_tensors in layer_outputs_tensors
        ])
        # The resulting shape of example_outputs is [num_layers, non_padded_length, hidden_size]
        # e.g., [32, 128, 4096]

        batch_layer_outputs.append(example_outputs)

    return batch_layer_outputs

In [8]:
import torch

def process_saved_responses_batch(responses_list, tokenizer, model):
    """
    Get layer activations for a batch of saved responses without generation.
    This version is corrected to be robust to any batch size, including 1.
    """
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    tokenized_batch = tokenizer(
        responses_list,
        padding=True,
        return_tensors="pt"
    ).to("cuda")

    input_ids = tokenized_batch.input_ids
    attention_mask = tokenized_batch.attention_mask

    layer_outputs_proxies = []
    with model.trace({"input_ids": input_ids, "attention_mask": attention_mask}) as tracer:
        for layer_idx in range(model.config.num_hidden_layers):
            layer_outputs_proxies.append(model.model.layers[layer_idx].output[0].save())

    layer_outputs_tensors = [p.value.cpu().detach().to(torch.float32) for p in layer_outputs_proxies]

    batch_final_outputs = []
    num_examples_in_batch = input_ids.shape[0]

    for i in range(num_examples_in_batch):
        current_attention_mask = attention_mask[i]
        padding_length = (current_attention_mask == 0).sum().item()

        # This list comprehension now contains the critical fix.
        # It checks the dimension of each layer_tensor before indexing.
        example_outputs_list = []
        for idx, layer_tensor in enumerate(layer_outputs_tensors):

            # --- THE CORE FIX IS HERE ---
            if layer_tensor.ndim == 3:
                # If tensor is 3D (batch, seq, hidden), use the batch index 'i'.
                sliced_tensor = layer_tensor[i, padding_length:, :]
            elif layer_tensor.ndim == 2:
                # If tensor is 2D (seq, hidden), it's a batch of 1. Ignore 'i'.
                sliced_tensor = layer_tensor[padding_length:, :]
            else:
                raise ValueError(f"Unexpected tensor dimension {layer_tensor.ndim} at layer {idx}")

            example_outputs_list.append(sliced_tensor)

        # Now stack the correctly sliced tensors.
        example_outputs = torch.stack(example_outputs_list)

        batch_final_outputs.append(example_outputs)

    return batch_final_outputs

# Train vectors
 - change to your labels, must reflect the data in the data path
 - This step requires a lot of gpu (i used 3 A100s) but it's fairly quick (around 60sec)

## This assumes that you have a dataset in this format - check data/enhanced_annotations.json
{
  "timestamp": "YYYYMMDD_HHMMSS",
  "model_name": "model-identifier-here",
  "include_emotional": true,
  "responses": [
    {
      "message": "Insert your prompt or reflective question here, e.g., 'You're facing a setback in your project. What does this tell you about your abilities and next steps?'",
      "response": "Insert the response here. This should be a thought process that might include doubts, realizations, problem-solving, and action planning. Keep it introspective and progressive.",
      "annotation": "[\"thought-type\"] Insert thought-level annotations here, mapping portions of the response to categories like 'anxious-thinking', 'normal-thinking', 'negative-attribution'."
    }
  ]


In [ ]:
import os

# 1. Define Constants and Load Model
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" # Use the specific HF name
LABELS = [
    "depressive-thinking", "anxious-thinking", "negative-attribution",
    "pessimistic-projection", "normal-thinking"
]
DATA_PATH = "data/enhanced_annotations.json"
OUTPUT_VECTORS_PATH = "results/vars/cognitive_mean_vectors.pt"

# Load the model wrapped in nnsight
model = LanguageModel(MODEL_NAME, device_map="cuda", torch_dtype=torch.bfloat16)
tokenizer = model.tokenizer
tokenizer.pad_token = tokenizer.eos_token # Standard practice for Llama

# 2. Load Data and Initialize Mean Vectors Dictionary
with open(DATA_PATH, 'r') as f:
    # Access the 'responses' key to get the list
    annotated_data = json.load(f)['responses']

mean_vectors = defaultdict(lambda: {
    'mean': torch.zeros(model.config.num_hidden_layers, model.config.hidden_size),
    'count': 0
})

# 3. Process Data in Batches to Calculate Means
batch_size = 1
for i in tqdm(range(0, len(annotated_data), batch_size)):
    batch = annotated_data[i:i+batch_size]

    full_responses = [item['response'] for item in batch]
    annotated_responses = [item['annotation'] for item in batch]

    # Use the NEW, corrected helper function
    batch_layer_outputs = process_saved_responses_batch(full_responses, tokenizer, model)

    for j, example_layer_outputs in enumerate(batch_layer_outputs):
        # --- DEBUGGING: Uncomment to check shapes ---
        # print(f"Shape of example_layer_outputs: {example_layer_outputs.shape}")
        # Expected: [32, seq_len, 4096]

        label_positions = get_label_positions(
            annotated_responses[j],
            full_responses[j],
            tokenizer
        )

        for label, positions in label_positions.items():
            if label not in LABELS: continue

            for start_token, end_token in positions:
                # Average over the sequence length dimension (dim=1)
                segment_vectors = example_layer_outputs[:, start_token:end_token].mean(dim=1)

                # --- DEBUGGING: Uncomment to check shapes ---
                # print(f"Shape of segment_vectors for label '{label}': {segment_vectors.shape}")
                # Expected: [32, 4096]

                count = mean_vectors[label]['count']
                current_mean = mean_vectors[label]['mean']

                # --- DEBUGGING: Uncomment to check shapes ---
                # if count == 0: # Check only on the first update for a label
                #    print(f"Shape of current_mean for label '{label}': {current_mean.shape}")
                #    # Expected: [32, 4096]

                # This line should now work without error
                mean_vectors[label]['mean'] = current_mean + (segment_vectors - current_mean) / (count + 1)
                mean_vectors[label]['count'] += 1

os.makedirs(os.path.dirname(OUTPUT_VECTORS_PATH), exist_ok=True)

# 4. Save the Raw Mean Vectors
# We save the raw means, not the final subtracted steering vectors
save_dict = {k: {'mean': v['mean'], 'count': v['count']} for k, v in mean_vectors.items()}
torch.save(save_dict, OUTPUT_VECTORS_PATH)
print(f"✅ Mean vectors saved to {OUTPUT_VECTORS_PATH}")

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

  0%|          | 0/44 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

100%|██████████| 44/44 [00:26<00:00,  1.66it/s]

✅ Mean vectors saved to results/vars/cognitive_mean_vectors.pt


Save/download the saved .pt file so that you can re-use on smaller gpu setup

# Steering

- Steering config needs to reflect labels and you need to set which layer to take the activations from + strenght of positive/negative influence of vectors

This can be run on 20ish gb of VRAM (L4 from colab is ok) for 8b model

In [39]:


# 1. Define Constants and Load Model
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" # Use the specific HF name
LABELS = [
    "depressive-thinking", "anxious-thinking", "negative-attribution",
    "pessimistic-projection", "normal-thinking"
]
DATA_PATH = "data/enhanced_annotations.json"
OUTPUT_VECTORS_PATH = "/content/COT-steering/results/vars/cognitive_mean_vectors.pt"

# Load the model wrapped in nnsight
model = LanguageModel(MODEL_NAME, device_map="cuda", torch_dtype=torch.bfloat16)
tokenizer = model.tokenizer
tokenizer.pad_token = tokenizer.eos_token # Standard practice for Llama

from utils import custom_generate_steering # The steering generation function
import matplotlib.pyplot as plt
import numpy as np

# 1. Define Steering Configuration
# Based on your analysis in Step 2, you create this config.
# Let's assume your analysis found layer 15 to be effective.
STEERING_CONFIG = {
    MODEL_NAME: {
        "depressive-thinking": {
            "vector_layer": 12,
            "pos_layers": [12],
            "neg_layers": [12],
            "pos_coefficient": 0.4,
            "neg_coefficient": 1
        },
        "anxious-thinking": {
            "vector_layer": 12,
            "pos_layers": [12],
            "neg_layers": [12],
            "pos_coefficient": 0.6,
            "neg_coefficient": 1.3
        },
        "negative-attribution": {
            "vector_layer": 12,
            "pos_layers": [12],
            "neg_layers": [12],
            "pos_coefficient": 1.3,
            "neg_coefficient": 1.3
        },
        "pessimistic-projection": {
            "vector_layer": 12,
            "pos_layers": [12],
            "neg_layers": [12],
            "pos_coefficient": 1.3,
            "neg_coefficient": 1.3
        },
        "normal-thinking": {
            "vector_layer": 12,
            "pos_layers": [12],
            "neg_layers": [12],
            "pos_coefficient": 1.3,
            "neg_coefficient": 1.3
        }
    }
}

# 2. Load Steering Vectors
# This time, we compute the final vectors (mean - baseline)
mean_vectors_data = torch.load(OUTPUT_VECTORS_PATH)
normal_thinking_mean = mean_vectors_data['normal-thinking']['mean']

feature_vectors = {}
for label in LABELS:
    if label != 'normal-thinking':
        feature_vectors[label] = mean_vectors_data[label]['mean'] - normal_thinking_mean
        # Optional: Normalize the vector (as done in the repo)
        for layer in range(model.config.num_hidden_layers):
            feature_vectors[label][layer] = (
                feature_vectors[label][layer]
                / feature_vectors[label][layer].norm()
                * normal_thinking_mean[layer].norm()
            )

# 3. Qualitative Test (Spot Check)
test_prompt = "My boss gave me some critical feedback on my latest report."
message = {"role": "user", "content": test_prompt}
input_ids = tokenizer.apply_chat_template([message], add_generation_prompt=True, return_tensors="pt").to("cuda")

print("--- Original Response ---")
# Generate with no steering
original_output = custom_generate_steering(model, tokenizer, input_ids, max_new_tokens=2000, label="none", feature_vectors=None, steering_config=STEERING_CONFIG[MODEL_NAME])
print(tokenizer.decode(original_output[0]))

print("\n--- Steered POSITIVE for Depressive Thinking ---")
# Steer towards depressive thinking
depressive_output = custom_generate_steering(model, tokenizer, input_ids, max_new_tokens=2000, label="anxious-thinking", feature_vectors=feature_vectors, steering_config=STEERING_CONFIG[MODEL_NAME], steer_positive=True)
print(tokenizer.decode(depressive_output[0]))

print("\n--- Steered NEGATIVE for Depressive Thinking (towards normal) ---")
# Steer away from depressive thinking
normal_output = custom_generate_steering(model, tokenizer, input_ids, max_new_tokens=2000, label="anxious-thinking", feature_vectors=feature_vectors, steering_config=STEERING_CONFIG[MODEL_NAME], steer_positive=False)
print(tokenizer.decode(normal_output[0]))

# 4. Quantitative Evaluation
# The original repo uses an API call to re-annotate and measure label frequency.
# A simpler, self-contained method is to measure the cosine similarity of the
# generated output's hidden state to the target vector.

# Quantitative Test Logic:
# a. Create a list of test prompts.
# b. For each prompt and each label (e.g., "anxious-thinking"):
#    i. Generate 3 responses: original, steered-positive, steered-negative.
#    ii. For each response, get its average hidden state at the steering layer (e.g., layer 16).
#    iii. Calculate the cosine similarity of this average hidden state with the 'anxious-thinking' steering vector.
# c. Collect all similarity scores.
# d. Create a bar plot showing the average cosine similarity for each condition.
#    - You expect the "steered-positive" bar to be the highest, and "steered-negative" to be the lowest.
#    - This provides quantitative proof that your steering is pushing the model's internal state in the intended direction.

--- Original Response ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<｜begin▁of▁sentence｜><｜User｜>My boss gave me some critical feedback on my latest report.<｜Assistant｜><think>
Okay, so my boss gave me some critical feedback on my latest report. Hmm, I need to figure out how to handle this. First, I should probably take a deep breath and not react right away. Getting emotional might not help. 

I remember reading somewhere that it's important to acknowledge the feedback without getting defensive. Maybe I should say something like, "I appreciate your feedback. I'll take it into consideration." That sounds calm and professional. 

But wait, why did my boss give me this feedback? Did I make a mistake or is there a misunderstanding? I should try to understand their perspective. Maybe I should ask for specific examples so I can address the issues better. Something like, "Could you give me more details about what you mean by 'clarity issues'?" That way, I know exactly what needs fixing.

I also need to think about how to improve. If the feedback is about com

## if running steering again eject model and clear cache

In [36]:
1/0  # break execution to clear traceback

ZeroDivisionError: division by zero

In [38]:
# At end of loop or after heavy GPU use:
# del model
import gc; gc.collect()
torch.cuda.empty_cache()